# Census 2011 Disability Dashboard - Data Processor

This notebook fetches data from your GitHub repository and generates JavaScript code for your HTML dashboard.

**No file uploads needed** - it reads directly from GitHub!

In [ ]:
import pandas as pd

print("=" * 80)
print("CENSUS 2011 DISABILITY DASHBOARD - DATA PROCESSOR")
print("=" * 80)
print()

## Step 1: Load Data from GitHub

In [ ]:
print("STEP 1: Loading Census data from GitHub...")
print("-" * 80)

github_url = "https://raw.githubusercontent.com/c-malhotra/c-malhotra.github.io/refs/heads/main/PC1/Census%20pwd%20data%20state.csv"

df = pd.read_csv(github_url, encoding='utf-8')

print(f"✓ Loaded {len(df):,} rows")
print(f"✓ Columns: {df.shape[1]}")
print()

## Step 2: Filter State-Level Totals

In [ ]:
print("STEP 2: Filtering to state-level totals...")
print("-" * 80)

df_states = df[
    (df['area_type'] == 'Total') &
    (df['Type of Household'] == 'Total') &
    (df['area_name'].str.contains('State -', case=False, na=False))
].copy()

df_states['state'] = df_states['area_name'].str.replace('State - ', '', case=False).str.strip()

print(f"✓ Found {len(df_states)} states/UTs")
print(f"✓ States: {', '.join(df_states['state'].head(5).tolist())}...")
print()

## Step 3: Calculate National Totals

In [ ]:
print("STEP 3: Calculating national totals...")
print("-" * 80)

national_total_pwd = int(df_states['Total disabled persons'].sum())
national_male_pwd = int(df_states['Total disabled males'].sum())
national_female_pwd = int(df_states['Total disabled females'].sum())
national_sex_ratio = int((national_female_pwd / national_male_pwd) * 1000)

print(f"Total PWDs: {national_total_pwd:,}")
print(f"Male: {national_male_pwd:,} ({(national_male_pwd/national_total_pwd*100):.1f}%)")
print(f"Female: {national_female_pwd:,} ({(national_female_pwd/national_total_pwd*100):.1f}%)")
print(f"Sex Ratio: {national_sex_ratio}:1000")
print()

## Step 4: Calculate Disability Type Distribution

In [ ]:
print("STEP 4: Calculating disability types...")
print("-" * 80)

disability_types = [
    ('Seeing', int(df_states['Persons_in seeing'].sum())),
    ('Movement', int(df_states['Persons_in movement'].sum())),
    ('Hearing', int(df_states['Persons_in hearing'].sum())),
    ('Other', int(df_states['Persons_any other'].sum())),
    ('Multiple', int(df_states['Persons_multiple disability'].sum())),
    ('Speech', int(df_states['Persons_in speech'].sum())),
    ('Mental Retardation', int(df_states['Persons_mental retardation'].sum())),
    ('Mental Illness', int(df_states['Persons_mental illness'].sum()))
]

disability_data = []
for name, count in disability_types:
    pct = round((count / national_total_pwd) * 100, 2)
    disability_data.append({'name': name, 'count': count, 'pct': pct})
    print(f"{name:20s}: {count:>10,} ({pct:5.2f}%)")

print()

## Step 5: Calculate State Prevalence

In [ ]:
print("STEP 5: Calculating state prevalence...")
print("-" * 80)

state_populations = {
    'ANDAMAN & NICOBAR ISLANDS': 379944,
    'ANDHRA PRADESH': 84665533,
    'ARUNACHAL PRADESH': 1382611,
    'ASSAM': 31169272,
    'BIHAR': 103804637,
    'CHANDIGARH': 1054686,
    'CHHATTISGARH': 25540196,
    'DADRA & NAGAR HAVELI': 342853,
    'DAMAN & DIU': 242911,
    'GOA': 1457723,
    'GUJARAT': 60383628,
    'HARYANA': 25353081,
    'HIMACHAL PRADESH': 6856509,
    'JAMMU & KASHMIR': 12548926,
    'JHARKHAND': 32966238,
    'KARNATAKA': 61130704,
    'KERALA': 33387677,
    'LAKSHADWEEP': 64429,
    'MADHYA PRADESH': 72597565,
    'MAHARASHTRA': 112372972,
    'MANIPUR': 2721756,
    'MEGHALAYA': 2964007,
    'MIZORAM': 1091014,
    'NAGALAND': 1980602,
    'NCT OF DELHI': 16753235,
    'ODISHA': 41947358,
    'PUDUCHERRY': 1244464,
    'PUNJAB': 27704236,
    'RAJASTHAN': 68621012,
    'SIKKIM': 607688,
    'TAMIL NADU': 72138958,
    'TRIPURA': 3671032,
    'UTTAR PRADESH': 199581477,
    'UTTARAKHAND': 10116752,
    'WEST BENGAL': 91347736
}

df_states['state_upper'] = df_states['state'].str.upper()
df_states['population'] = df_states['state_upper'].map(state_populations)
df_states['prevalence'] = ((df_states['Total disabled persons'] / df_states['population']) * 100).round(2)

df_states = df_states.sort_values('prevalence', ascending=False)

print(f"✓ Calculated for {len(df_states)} states")
print()
print("Top 10 states by prevalence:")
for idx, row in df_states.head(10).iterrows():
    print(f"  {row['state']:30s}: {row['prevalence']:5.2f}% ({int(row['Total disabled persons']):>10,} PWDs)")
print()

## Step 6: Generate JavaScript Code for HTML Dashboard

**Copy the output below and paste it into your HTML file!**

In [ ]:
print("=" * 80)
print("JAVASCRIPT CODE FOR HTML DASHBOARD")
print("=" * 80)
print()
print("Copy everything between the lines below:")
print()
print("-" * 80)
print()

# State data
print("const stateData = [")
for _, row in df_states.iterrows():
    print(f"  {{ state: '{row['state']}', prevalence: {row['prevalence']}, pwd_pop: {int(row['Total disabled persons'])} }},")
print("];")
print()

# All states for search
print("const allIndianStates = [")
all_states = [
    'Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh',
    'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jharkhand',
    'Karnataka', 'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur',
    'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Punjab',
    'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura',
    'Uttar Pradesh', 'Uttarakhand', 'West Bengal', 'Jammu & Kashmir',
    'Delhi', 'Puducherry', 'Ladakh', 'Chandigarh', 'Andaman and Nicobar Islands',
    'Dadra and Nagar Haveli and Daman and Diu', 'Lakshadweep'
]
for state in all_states:
    print(f"  '{state}',")
print("];")
print()

# Disability types
colors = {
    'Seeing': '#9333ea', 'Movement': '#7c3aed', 'Hearing': '#a855f7',
    'Other': '#c084fc', 'Multiple': '#d8b4fe', 'Speech': '#e9d5ff',
    'Mental Retardation': '#ec4899', 'Mental Illness': '#f9a8d4'
}

disability_data_sorted = sorted(disability_data, key=lambda x: x['pct'], reverse=True)

print("const disabilityTypes = [")
for d in disability_data_sorted:
    color = colors.get(d['name'], '#c084fc')
    print(f"  {{ name: '{d['name']}', pct: {d['pct']}, count: {d['count']}, color: '{color}' }},")
print("];")
print()

# National data
print("const nationalData = {")
print(f"  total_pwd: {national_total_pwd},")
print(f"  male_pwd: {national_male_pwd},")
print(f"  female_pwd: {national_female_pwd},")
print(f"  sex_ratio: {national_sex_ratio},")
print(f"  national_prevalence: 2.21,")
print(f"  rural_pct: 69,")
print(f"  urban_pct: 31,")
print(f"  homeless: 68000")
print("};")
print()

print("-" * 80)
print()
print("=" * 80)
print("✅ COMPLETE - Paste the code above into your HTML file")
print("=" * 80)